In [3]:
%load_ext autoreload
# %autoreload 2

In [4]:
import logging
from sys import stdout
from datetime import datetime
import numpy as np
import os

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)

logging.info(f"{os.getlogin()}| {datetime.now()}")

[02-06 12:48:39][INFO] riamu| 2025-02-06 12:48:39.616351


In [5]:
# ####################################################################
def gauss_jordan(Ab: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante el método de Gauss-Jordan.

    ## Parameters

    ``Ab``: matriz aumentada del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(n+1), donde n es el número de incógnitas.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """
    if not isinstance(Ab, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        Ab = np.array(Ab, dtype=float)
    assert Ab.shape[0] == Ab.shape[1] - 1, "La matriz A debe ser de tamaño n-by-(n+1)."
    n = Ab.shape[0]

    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if Ab[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux

        # --- Eliminación: loop por fila
        for j in range(n):
            if i == j:
                continue
            m = Ab[j, i] / Ab[i, i]
            Ab[j, i:] = Ab[j, i:] - m * Ab[i, i:]

        logging.info(f"\n{Ab}")

    if Ab[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

    # --- Sustitución hacia atrás
    solucion = np.zeros(n)

    for i in range(n - 1, -1, -1):
        solucion[i] = Ab[i, -1] / Ab[i, i]

    return solucion

In [6]:
%autoreload 2
from src import matriz_aumentada

A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
b = [1, -2, 3, 4]

Ab = matriz_aumentada(A, b)
gauss_jordan(Ab)

[02-06 12:48:49][INFO] 2025-02-06 12:48:49.979483
[02-06 12:48:49][INFO] riamu| 2025-02-06 12:48:49.981835
[02-06 12:48:49][INFO] 2025-02-06 12:48:49.984343
[02-06 12:48:49][INFO] riamu| 2025-02-06 12:48:49.986862
[02-06 12:48:49][INFO] 
[[ 1.  2.  3.  4.  1.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0. -1. -3. -6.  0.]]
[02-06 12:48:49][INFO] 
[[ 1.  0.  3.  6.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0. -3. -7. -4.]]
[02-06 12:48:49][INFO] 
[[ 1.  0.  0. -3.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0.  0.  2. -4.]]
[02-06 12:48:49][INFO] 
[[ 1.  0.  0.  0.  3.]
 [ 0.  1.  0.  0. -6.]
 [ 0.  0. -1.  0. -6.]
 [ 0.  0.  0.  2. -4.]]


array([ 3., -6.,  6., -2.])

# Resolver

In [15]:
# ####################################################################
def inv_matrix(A: np.ndarray) -> np.ndarray:
    """Inversión de una matriz cuadrada mediante método de Gauss-Jordan.
    ## Parameters
    ``A``: matriz cuadrada de tamaño n x n.

    ## Return
    ``A_inv``: matriz inversa de A.
    """

    
    if not isinstance(A, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        A = np.array(A, dtype=float)
    assert A.shape[0] == A.shape[1], "La matriz A debe ser cuadrada."
    n = A.shape[0]

    # Crear una matriz aumentada con la matriz identidad
    Ab = np.hstack([A, np.eye(n)])

    # Aplicar el método de Gauss-Jordan
    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if Ab[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux

        # --- Eliminación: loop por fila
        for j in range(n):
            if i == j:
                continue
            m = Ab[j, i] / Ab[i, i]
            Ab[j, i:] = Ab[j, i:] - m * Ab[i, i:]

        logging.info(f"\n{Ab}")

    if Ab[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

    # Extraer la matriz inversa
    A_inv = Ab[:, n:]

    return A_inv
    # COMPLETAR
    # Debe basarse en la función gauss_jordan
    pass

## Ejemplos
* Ejemplo 1

In [16]:
# La matriz A =
A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
# tiene como inversa
# A_inv =[[ 0.5, -0.5, -1.5,  1.5],
#        [-0.5,  1.5, -1.5,  0.5],
#        [-1.5, -1.5,  3.5, -1.5],
#        [ 1.5,  0.5, -1.5,  0.5]]
inv_matrix(A)

[02-06 12:53:57][INFO] 
[[ 1.  2.  3.  4.  1.  0.  0.  0.]
 [ 0.  1.  0. -1. -2.  1.  0.  0.]
 [ 0.  0. -1. -3. -3.  0.  1.  0.]
 [ 0. -1. -3. -6. -4.  0.  0.  1.]]
[02-06 12:53:57][INFO] 
[[ 1.  0.  3.  6.  5. -2.  0.  0.]
 [ 0.  1.  0. -1. -2.  1.  0.  0.]
 [ 0.  0. -1. -3. -3.  0.  1.  0.]
 [ 0.  0. -3. -7. -6.  1.  0.  1.]]
[02-06 12:53:57][INFO] 
[[ 1.  0.  0. -3. -4. -2.  3.  0.]
 [ 0.  1.  0. -1. -2.  1.  0.  0.]
 [ 0.  0. -1. -3. -3.  0.  1.  0.]
 [ 0.  0.  0.  2.  3.  1. -3.  1.]]
[02-06 12:53:57][INFO] 
[[ 1.   0.   0.   0.   0.5 -0.5 -1.5  1.5]
 [ 0.   1.   0.   0.  -0.5  1.5 -1.5  0.5]
 [ 0.   0.  -1.   0.   1.5  1.5 -3.5  1.5]
 [ 0.   0.   0.   2.   3.   1.  -3.   1. ]]


array([[ 0.5, -0.5, -1.5,  1.5],
       [-0.5,  1.5, -1.5,  0.5],
       [ 1.5,  1.5, -3.5,  1.5],
       [ 3. ,  1. , -3. ,  1. ]])

* Ejemplo 2

In [17]:
# La matriz A =
A = [
    [4, 4, 5, 1],
    [3, 4, 2, 2],
    [2, 1, 4, 1],
    [3, 2, 5, 4],
]
# tiene como inversa
# A_inv =[[-34.,  31.,  52., -20.],
#         [ 19., -17., -29.,  11.],
#         [ 12., -11., -18.,   7.],
#         [  1.,  -1.,  -2.,   1.]]
inv_matrix(A)

[02-06 12:54:06][INFO] 
[[ 2.   1.   4.   1.   0.   0.   1.   0. ]
 [ 0.   2.5 -4.   0.5  0.   1.  -1.5  0. ]
 [ 0.   2.  -3.  -1.   1.   0.  -2.   0. ]
 [ 0.   0.5 -1.   2.5  0.   0.  -1.5  1. ]]
[02-06 12:54:06][INFO] 
[[  2.    0.    6.   -4.    0.    0.    4.   -2. ]
 [  0.    0.5  -1.    2.5   0.    0.   -1.5   1. ]
 [  0.    0.    1.  -11.    1.    0.    4.   -4. ]
 [  0.    0.    1.  -12.    0.    1.    6.   -5. ]]


[02-06 12:54:06][INFO] 
[[  2.    0.    0.   62.   -6.    0.  -20.   22. ]
 [  0.    0.5   0.   -8.5   1.    0.    2.5  -3. ]
 [  0.    0.    1.  -11.    1.    0.    4.   -4. ]
 [  0.    0.    0.   -1.   -1.    1.    2.   -1. ]]
[02-06 12:54:06][INFO] 
[[  2.    0.    0.    0.  -68.   62.  104.  -40. ]
 [  0.    0.5   0.    0.    9.5  -8.5 -14.5   5.5]
 [  0.    0.    1.    0.   12.  -11.  -18.    7. ]
 [  0.    0.    0.   -1.   -1.    1.    2.   -1. ]]


array([[-68. ,  62. , 104. , -40. ],
       [  9.5,  -8.5, -14.5,   5.5],
       [ 12. , -11. , -18. ,   7. ],
       [ -1. ,   1. ,   2. ,  -1. ]])

## Ejercicios

* Ejercicio 1

In [18]:
A = [[2, -3], [-1, 1]]
inv_matrix(A)

[02-06 12:54:10][INFO] 
[[-1.  1.  0.  1.]
 [ 0. -1.  1.  2.]]
[02-06 12:54:10][INFO] 
[[-1.  0.  1.  3.]
 [ 0. -1.  1.  2.]]


array([[1., 3.],
       [1., 2.]])

* Ejercicio 2

In [19]:
A = [
    [4, 0, 0, 5],
    [1, 0, 4, 0],
    [3, 4, 1, 3],
    [1, 3, 3, 0],
]
inv_matrix(A)

[02-06 12:54:42][INFO] 
[[  1.   0.   4.   0.   0.   1.   0.   0.]
 [  0.   0. -16.   5.   1.  -4.   0.   0.]
 [  0.   4. -11.   3.   0.  -3.   1.   0.]
 [  0.   3.  -1.   0.   0.  -1.   0.   1.]]
[02-06 12:54:42][INFO] 
[[  1.           0.           4.           0.           0.
    1.           0.           0.        ]
 [  0.           3.          -1.           0.           0.
   -1.           0.           1.        ]
 [  0.           0.          -9.66666667   3.           0.
   -1.66666667   1.          -1.33333333]
 [  0.           0.         -16.           5.           1.
   -4.           0.           0.        ]]
[02-06 12:54:42][INFO] 
[[ 1.          0.          0.          1.24137931  0.          0.31034483
   0.4137931  -0.55172414]
 [ 0.          3.          0.         -0.31034483  0.         -0.82758621
  -0.10344828  1.13793103]
 [ 0.          0.         -9.66666667  3.          0.         -1.66666667
   1.         -1.33333333]
 [ 0.          0.          0.          0.034482

array([[ -36.        ,   45.        ,   60.        ,  -80.        ],
       [   9.        ,  -12.        ,  -15.        ,   21.        ],
       [ -87.        ,  106.33333333,  145.        , -193.33333333],
       [   1.        ,   -1.24137931,   -1.65517241,    2.20689655]])

* Ejercicio 3

In [76]:
A = [
    [0, 0, 0, 0, 0, 0, 1, -1],
    [0, 1, -1, 1, 0, -1, 0, 1],
    [-1, -1, 0, 0, 2, 1, 0, 0],
    [-1, -1, -1, 1, 2, 0, 0, 1],
    [-1, 1, 1, 0, -1, -1, 0, 2],
    [0, 1, 0, 0, -1, -1, 0, 0],
    [1, -1, -1, 1, 2, 1, 0, 2],
    [2, 0, 0, 0, 0, 1, 2, 0],
]
inv_matrix(A)

* Ejercicio 4

In [77]:
A = [
    [1, 0, 0, 0, -1, 0, 0, -1, 1, -1],
    [1, 1, 0, -1, -1, 1, 0, 0, 1, -1],
    [-1, 0, -1, 0, 0, 0, -1, 1, 0, 0],
    [0, 0, -1, 0, -1, -1, 1, 0, 1, 0],
    [-1, 0, 0, -1, 1, 1, 1, 1, 0, -1],
    [1, 0, 0, 1, -1, -1, -1, 1, -1, 0],
    [1, 1, 1, 0, 1, 0, -1, -1, -1, 1],
    [1, 1, 1, 1, 0, 0, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 0, -1, -1, 0, 0],
    [0, 0, -1, -1, -1, 0, 1, 1, 1, -1],
]
inv_matrix(A)